# Maersk Triangulation

## Setup

### Imports & API

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import requests
import concurrent.futures


## Load Files

In [2]:
# Load one of the Excel files
fro_worklist_path = "C://Users//Ani//Downloads//FRO worklist dump - CMU.xlsx"
fro_list_path = 'C://Users//Ani//Downloads//FRO list - CMU.xlsx'
geocodes_list_path = 'C://Users//Ani//Downloads//Geocodes list - CMU.xlsx'
location_list_path = 'C://Users//Ani//Downloads//Location list - CMU.xlsx'

# Read the data from the Excel file
fro_worklist_data = pd.read_excel(fro_worklist_path)
fro_list_data = pd.read_excel(fro_list_path)
geocodes_list_data = pd.read_excel(geocodes_list_path)
location_list_data = pd.read_excel(location_list_path)

In [7]:
# Set your Azure Maps subscription key
api_key = "dtOo2U8CwwZkR4GAnj06HD3zzU5-yS_dNtKxDkOrtF4"

In [8]:
def get_route(api_key, coordinates):
    url = "https://atlas.microsoft.com/route/directions/json"
    # Ensure coordinates are in [longitude, latitude] format for the query parameter
    coords_str = ":".join([f"{lon},{lat}" for lat, lon in coordinates])
    params = {
        "api-version": "1.0",
        "subscription-key": api_key,
        "query": coords_str,
        'travelMode': 'truck'
    }
    response = requests.get(url, params=params)
    # Check response status before returning JSON to handle possible errors
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code, "message": response.text}

def fetch_routes_in_batch(api_key, routes):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Submitting batch routing requests
        future_to_route = {executor.submit(get_route, api_key, route): route for route in routes}
        for future in concurrent.futures.as_completed(future_to_route):
            route = future_to_route[future]
            try:
                data = future.result()
            except Exception as exc:
                print(f"{route} generated an exception: {exc}")
            else:
                results.append(data)
    return results

# Example usage with your API key

routes = [
    # Ensure each pair within routes is in [longitude, latitude] order
    [(-122.4783, 37.8199), (-115.1728, 36.1147), (-118.2437, 34.0522)],  # Route 1: Golden Gate Bridge to Las Vegas
    [(-74.0060, 40.7128), (-118.2437, 34.0522)]  # Route 2: New York City to Los Angeles
   # [(-115.1728, 36.1147), (-122.4783, 37.8199)]]  # Route 3: Golden Gate Bridge to Los Angeles
]
batch_results = fetch_routes_in_batch(api_key, routes)

def extract_route_summary(batch_results):
    summaries = []
    for result in batch_results:
        if 'routes' in result:
            for route in result['routes']:
                if 'summary' in route:
                    summary = route['summary']
                    summaries.append({
                        'lengthInMeters': summary.get('lengthInMeters'),
                        'travelTimeInSeconds': summary.get('travelTimeInSeconds')
                    })
    return summaries

route_summaries = extract_route_summary(batch_results)
print(route_summaries)

[{'lengthInMeters': 4609225, 'travelTimeInSeconds': 150205}, {'lengthInMeters': 1381113, 'travelTimeInSeconds': 56355}]


## Main

In [8]:
imports = fro_worklist_data[fro_worklist_data['Traffic Direction'] == 'Import']
exports = fro_worklist_data[fro_worklist_data['Traffic Direction'] == 'Export']

def build_matching_dataframe(imports, exports):


    # Convert 'Appointment Date/Time' to datetime format if not already
    imports['Appointment Date/Time'] = pd.to_datetime(imports['Appointment Date/Time'], errors='coerce')
    exports['Appointment Date/Time'] = pd.to_datetime(exports['Appointment Date/Time'], errors='coerce')

    matching_details_list = []
    
    for _, import_row in imports.iterrows():
        import_date = import_row['Appointment Date/Time'].date()
        
        potential_exports = exports[
            (exports['Source City'] == import_row['Source City']) &
            (exports['Means of Transport'] == import_row['Means of Transport']) &
            (exports['Equipment Group'] == import_row['Equipment Group']) &
            (exports['Equipment Type'] == import_row['Equipment Type']) &
            exports['Appointment Date/Time'].dt.date.isin([import_date, import_date + timedelta(days=1)])
        ]
        
        for _, export_row in potential_exports.iterrows():
            matching_details_list.append({
                "Source City": import_row['Source City'],
                "Import Customer City": import_row['Customer City Name'],
                "Export Customer City": export_row['Customer City Name'],
                "Import Appointment Date/Time": import_row['Appointment Date/Time'],
                "Export Appointment Date/Time": export_row['Appointment Date/Time']
            })
    
    return pd.DataFrame(matching_details_list)

# Assuming 'imports' and 'exports' are defined DataFrames
matching_df = build_matching_dataframe(imports, exports)
matching_df.head()


c:\Users\Ani\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\Ani\AppData\Local\Temp\ipykernel_856\504144714.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imports['Appointment Date/Time'] = pd.to_datetime(imports['Appointment Date/Time'], errors='coerce')
c:\Users\Ani\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aw

,Source City,Import Customer City,Export Customer City,Import Appointment Date/Time,Export Appointment Date/Time
0,Aarhus,Horsens,Randers,2024-01-02 06:00:00,2024-01-02 09:00:00
1,Aarhus,Horsens,Ringkoebing,2024-01-02 06:00:00,2024-01-02 09:30:00
2,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 07:00:00
3,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 08:00:00
4,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 13:00:00


In [14]:
def preprocess_input_data(api_key, input_data):
    unique_routes = []
    for idx, entry in enumerate(input_data):
        source_coords = geocode_city(api_key, entry['Source City'].split(';')[-1].strip())
        import_coords = geocode_city(api_key, entry['Import Customer City'].split(';')[-1].strip())
        export_coords = geocode_city(api_key, entry['Export Customer City'].split(';')[-1].strip())


        # Append route information as dictionaries including descriptive text
        unique_routes.append([source_coords, import_coords, source_coords])
        unique_routes.append([source_coords, export_coords, source_coords])
        unique_routes.append([source_coords, import_coords, export_coords, source_coords])
        unique_routes.append([import_coords, export_coords])

    return unique_routes

geocode_cache = {}  # Cache for geocoding results

def geocode_city(api_key, city_name):

    if city_name in geocode_cache:
        return geocode_cache[city_name]  # Return cached result if available
    
    # Geocoding API call (simplified example)
    url = f"https://atlas.microsoft.com/search/address/json?api-version=1.0&subscription-key={api_key}&query={city_name}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and data['results']:
            position = data['results'][0]['position']
            coordinates = (position['lon'], position['lat'])
            geocode_cache[city_name] = coordinates  # Cache result
            return coordinates
    return None, None  # Handle errors or missing data appropriately
    
routes = preprocess_input_data(api_key, matching_df.to_dict(orient='records'))
len(routes)

18396

In [15]:
batch_results = fetch_routes_in_batch(api_key, routes)

def extract_route_summary(batch_results):
    summaries = []
    for result in batch_results:
        if 'routes' in result:
            for route in result['routes']:
                if 'summary' in route:
                    summary = route['summary']
                    summaries.append({
                        'lengthInMeters': summary.get('lengthInMeters'),
                        'travelTimeInSeconds': summary.get('travelTimeInSeconds')
                    })
    return summaries

route_summaries = extract_route_summary(batch_results)
print(route_summaries)

[{'lengthInMeters': 81209, 'travelTimeInSeconds': 3602}, {'lengthInMeters': 101807, 'travelTimeInSeconds': 5079}, {'lengthInMeters': 101807, 'travelTimeInSeconds': 5079}, {'lengthInMeters': 101807, 'travelTimeInSeconds': 5079}, {'lengthInMeters': 79162, 'travelTimeInSeconds': 4200}, {'lengthInMeters': 172697, 'travelTimeInSeconds': 8243}, {'lengthInMeters': 273942, 'travelTimeInSeconds': 12482}, {'lengthInMeters': 279830, 'travelTimeInSeconds': 12731}, {'lengthInMeters': 102363, 'travelTimeInSeconds': 4820}, {'lengthInMeters': 319600, 'travelTimeInSeconds': 14790}, {'lengthInMeters': 102363, 'travelTimeInSeconds': 4820}, {'lengthInMeters': 101807, 'travelTimeInSeconds': 5079}, {'lengthInMeters': 273942, 'travelTimeInSeconds': 12483}, {'lengthInMeters': 294480, 'travelTimeInSeconds': 13781}, {'lengthInMeters': 101807, 'travelTimeInSeconds': 5079}, {'lengthInMeters': 102363, 'travelTimeInSeconds': 4821}, {'lengthInMeters': 101807, 'travelTimeInSeconds': 5079}, {'lengthInMeters': 294480, 

the code below proves no travel time takes more than a day

In [53]:
count = sum(summary['travelTimeInSeconds'] > 86400 for summary in route_summaries)
count


0

In [45]:
len(route_summaries)

18394

In [27]:
def add_route_summary_columns(matching_df, route_summaries):
    # Initialize columns with default values
    matching_df['Import Time'] = 0
    matching_df['Import Distance'] = 0
    matching_df['Export Time'] = 0
    matching_df['Export Distance'] = 0
    matching_df['Triangulation Time'] = 0
    matching_df['Triangulation Distance'] = 0
    matching_df['Import Export Time'] = 0
    matching_df['Import Export Distance'] = 0
    
    # Iterate through matching_df with step size of 4 in route_summaries
    for idx in range(0, len(matching_df)):
        # Calculate the starting index in route_summaries for the current row
        summary_idx = idx * 4

        # Check if there are enough entries in route_summaries for the current row
        if summary_idx + 3 < len(route_summaries):
            matching_df.at[idx, 'Import Time'] = route_summaries[summary_idx]['travelTimeInSeconds']
            matching_df.at[idx, 'Import Distance'] = route_summaries[summary_idx]['lengthInMeters']
            matching_df.at[idx, 'Export Time'] = route_summaries[summary_idx + 1]['travelTimeInSeconds']
            matching_df.at[idx, 'Export Distance'] = route_summaries[summary_idx + 1]['lengthInMeters']
            matching_df.at[idx, 'Triangulation Time'] = route_summaries[summary_idx + 2]['travelTimeInSeconds']
            matching_df.at[idx, 'Triangulation Distance'] = route_summaries[summary_idx + 2]['lengthInMeters']
            matching_df.at[idx, 'Import Export Time'] = route_summaries[summary_idx + 3]['travelTimeInSeconds']
            matching_df.at[idx, 'Import Export Distance'] = route_summaries[summary_idx + 3]['lengthInMeters']
        else:
            # If not enough entries, print the row number and insert 0s
            print(f"Row {idx + 1} is missing route summaries; inserting default values.")
            # Default values are already set during initialization, so no further action is needed for insertion

    return matching_df


updated_df = add_route_summary_columns(matching_df, route_summaries)

Row 4599 is missing route summaries; inserting default values.


In [28]:
updated_df

,Source City,Import Customer City,Export Customer City,Import Appointment Date/Time,Export Appointment Date/Time,Import Time,Import Distance,Export Time,Export Distance,Triangulation Time,Triangulation Distance,Import Export Time,Import Export Distance
0,Aarhus,Horsens,Randers,2024-01-02 06:00:00,2024-01-02 09:00:00,3602,81209,5079,101807,5079,101807,5079,101807
1,Aarhus,Horsens,Ringkoebing,2024-01-02 06:00:00,2024-01-02 09:30:00,4200,79162,8243,172697,12482,273942,12731,279830
2,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 07:00:00,4820,102363,14790,319600,4820,102363,5079,101807
3,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 08:00:00,12483,273942,13781,294480,5079,101807,4821,102363
4,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 13:00:00,5079,101807,13781,294480,5875,126602,13780,294480
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4594,Aarhus,Silkeborg,Padborg; Aarhus,2024-02-14 10:00:00,2024-02-14 10:30:00,0,0,4724,95500,2329,47277,4724,95500
4595,Aarhus,Silkeborg,Padborg; Aarhus,2024-02-14 10:00:00,2024-02-14 09:30:00,0,0,4724,95500,4724,95500,0,0
4596,Aarhus,Silkeborg,Padborg; Aarhus,2024-02-14 10:00:00,2024-02-14 11:30:00,2329,47277,0,0,4724,95500,4724,95500
4597,Aarhus,Silkeborg,Holstebro; Aarhus,2024-02-14 10:00:00,2024-02-14 09:00:00,2329,47277,4724,95500,2329,47277,4724,95500


however, we see here that there now times that suddenly exceed a day

In [41]:
filtered_df = updated_df[(updated_df['Import Time'] > 86400) | (updated_df['Export Time'] > 86400) | (updated_df['Triangulation Time'] > 86400) | (updated_df['Import Export Time'] > 86400)]
filtered_df

,Source City,Import Customer City,Export Customer City,Import Appointment Date/Time,Export Appointment Date/Time,Import Time,Import Distance,Export Time,Export Distance,Triangulation Time,Triangulation Distance,Import Export Time,Import Export Distance
0,Aarhus,Horsens,Randers,2024-01-02 06:00:00,2024-01-02 09:00:00,3602,81209,5079,101807,5079,101807,5079000000000,101807
1,Aarhus,Horsens,Ringkoebing,2024-01-02 06:00:00,2024-01-02 09:30:00,4200,79162,8243,172697,12482,273942,12731000000000,279830
2,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 07:00:00,4820,102363,14790,319600,4820,102363,5079000000000,101807
3,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 08:00:00,12483,273942,13781,294480,5079,101807,4821000000000,102363
4,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 13:00:00,5079,101807,13781,294480,5875,126602,13780000000000,294480
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4592,Aarhus,Silkeborg,Holstebro; Aarhus,2024-02-14 10:00:00,2024-02-14 08:00:00,4724,95500,3909,92768,4724,95500,4724000000000,95500
4593,Aarhus,Silkeborg,Svenstrup,2024-02-14 10:00:00,2024-02-14 06:30:00,0,0,12272,284531,11930,283822,2329000000000,47277
4594,Aarhus,Silkeborg,Padborg; Aarhus,2024-02-14 10:00:00,2024-02-14 10:30:00,0,0,4724,95500,2329,47277,4724000000000,95500
4596,Aarhus,Silkeborg,Padborg; Aarhus,2024-02-14 10:00:00,2024-02-14 11:30:00,2329,47277,0,0,4724,95500,4724000000000,95500


In [37]:
def create_triangulation_potentials_dfs(df):
    # Ensure "Import Export Time" is treated as numeric seconds
    df['Import Export Time'] = pd.to_numeric(df['Import Export Time'], errors='coerce')

    # Initialize empty DataFrames for triangulations and potentials
    triangulations = pd.DataFrame(columns=df.columns)
    potentials = pd.DataFrame(columns=df.columns)
    
    for idx, row in df.iterrows():
        # Calculate time difference between Export and Import Appointments in seconds
        time_diff = (pd.to_datetime(row['Export Appointment Date/Time']) - pd.to_datetime(row['Import Appointment Date/Time'])).total_seconds()
        
        # Check if the condition for triangulation is met
        if time_diff > row['Import Export Time']:
            triangulations = triangulations.append(row, ignore_index=True)
        else:
            potentials = potentials.append(row, ignore_index=True)
    
    return triangulations, potentials

# Example usage
# Ensure that updated_df is defined and contains the correct columns before running this
triangulations_df, potentials_df = create_triangulation_potentials_dfs(updated_df)

C:\Users\Ani\AppData\Local\Temp\ipykernel_856\4054532495.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  potentials = potentials.append(row, ignore_index=True)
C:\Users\Ani\AppData\Local\Temp\ipykernel_856\4054532495.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  triangulations = triangulations.append(row, ignore_index=True)


In [38]:
triangulations_df

,Source City,Import Customer City,Export Customer City,Import Appointment Date/Time,Export Appointment Date/Time,Import Time,Import Distance,Export Time,Export Distance,Triangulation Time,Triangulation Distance,Import Export Time,Import Export Distance
0,Aarhus,Kolding,Svenstrup,2024-01-02 08:00:00,2024-01-02 09:00:00,8661,197895,11909,287483,16588,399641,0,0
1,Aarhus,Hammel,Svenstrup,2024-01-02 07:00:00,2024-01-02 08:00:00,4457,59821,11909,287483,12922,260522,0,0
2,Aarhus,Brabrand,Svenstrup,2024-01-02 07:00:00,2024-01-02 08:00:00,11927,283822,1925,20915,12233,285315,0,0
3,Aarhus,Brabrand,Esbjerg,2024-01-02 07:00:00,2024-01-02 08:00:00,14076,345010,1925,20915,13280,336586,0,0
4,Aarhus,Brabrand,Esbjerg,2024-01-02 07:00:00,2024-01-02 08:00:00,13280,336586,1925,20915,13265,336586,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Aarhus,Brabrand,Holstebro; Aarhus,2024-02-14 07:00:00,2024-02-14 09:00:00,1920,20915,1920,20915,0,0,0,0
205,Aarhus,Brabrand,Holstebro,2024-12-02 07:00:00,2024-12-02 09:00:00,8270,190881,11135,249445,4637,113984,0,0
206,Aarhus,Brabrand,Holstebro; Aarhus,2024-02-14 07:00:00,2024-02-14 08:00:00,1920,20915,1920,20915,1920,20915,0,0
207,Aarhus,Brabrand,Padborg; Aarhus,2024-02-14 07:00:00,2024-02-14 10:30:00,1920,20915,11930,283822,1920,20915,0,0


In [39]:
potentials_df

,Source City,Import Customer City,Export Customer City,Import Appointment Date/Time,Export Appointment Date/Time,Import Time,Import Distance,Export Time,Export Distance,Triangulation Time,Triangulation Distance,Import Export Time,Import Export Distance
0,Aarhus,Horsens,Randers,2024-01-02 06:00:00,2024-01-02 09:00:00,3602,81209,5079,101807,5079,101807,5079000000000,101807
1,Aarhus,Horsens,Ringkoebing,2024-01-02 06:00:00,2024-01-02 09:30:00,4200,79162,8243,172697,12482,273942,12731000000000,279830
2,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 07:00:00,4820,102363,14790,319600,4820,102363,5079000000000,101807
3,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 08:00:00,12483,273942,13781,294480,5079,101807,4821000000000,102363
4,Aarhus,Horsens,Skjern,2024-01-02 06:00:00,2024-01-02 13:00:00,5079,101807,13781,294480,5875,126602,13780000000000,294480
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4385,Aarhus,Silkeborg,Padborg; Aarhus,2024-02-14 10:00:00,2024-02-14 10:30:00,0,0,4724,95500,2329,47277,4724000000000,95500
4386,Aarhus,Silkeborg,Padborg; Aarhus,2024-02-14 10:00:00,2024-02-14 09:30:00,0,0,4724,95500,4724,95500,0,0
4387,Aarhus,Silkeborg,Padborg; Aarhus,2024-02-14 10:00:00,2024-02-14 11:30:00,2329,47277,0,0,4724,95500,4724000000000,95500
4388,Aarhus,Silkeborg,Holstebro; Aarhus,2024-02-14 10:00:00,2024-02-14 09:00:00,2329,47277,4724,95500,2329,47277,4724000000000,95500
